In [11]:
# Load Silver & Preview

df_silver = spark.table("silver_clean_trips")

print(f"Silver rows available: {df_silver.count():,}")
display(df_silver.limit(5))


StatementMeta(, 71c27480-8382-406f-a870-d2c3a3b36a18, 13, Finished, Available, Finished)

Silver rows available: 942,749


SynapseWidget(Synapse.DataFrame, 7e238f0c-d02b-40b3-95d0-84961672d666)

In [12]:
# Create Main Gold Table (Hourly Demand by Zone)
from pyspark.sql.functions import date_trunc, col,count, sum, avg, round

df_gold = df_silver \
          .withColumn("pickup_hour",date_trunc("hour",col("pickup_datetime")))\
          .groupBy("PULocationID","pickup_hour")\
          .agg(
                count("*").alias("trip_count"),
                round(sum("total_amount"), 2).alias("total_revenue"),
                round(avg("trip_distance"), 2).alias("avg_distance"),
                round(avg("tip_amount"), 2).alias("avg_tip"),
                round(avg("fare_amount"), 2).alias("avg_fare")
            ) \
            .orderBy("pickup_hour", ascending=False)

# Save as managed Delta table (Gold)
df_gold.write.format("delta").mode("overwrite").saveAsTable("nyc_hourly_zone_insights")

print("Gold table created: nyc_hourly_zone_insights")
display(df_gold.limit(20))

StatementMeta(, 71c27480-8382-406f-a870-d2c3a3b36a18, 14, Finished, Available, Finished)

Gold table created: nyc_hourly_zone_insights


SynapseWidget(Synapse.DataFrame, 42df1707-b318-48a7-9510-108503c7f6f5)

In [13]:
# Gold Metric : Daily Revenue
from pyspark.sql.functions import col,count, sum,year,month,dayofmonth

daily_revenue = (
    df_silver
    .withColumn("year", year("pickup_datetime"))
    .withColumn("month", month("pickup_datetime"))
    .withColumn("day", dayofmonth("pickup_datetime"))
    .groupBy("year", "month", "day")
    .agg(
        sum("fare_amount").alias("total_revenue"),
        count("*").alias("total_trips")
    )
)
# Save as managed Delta table (Gold)
daily_revenue.write.format("delta").mode("overwrite").partitionBy("year","month").saveAsTable("nyc_daily_revenue")

print("Gold table created: nyc_daily_revenue")
print(f"Rows after cleaning: {daily_revenue.count():,}")   
display(daily_revenue.limit(20))

StatementMeta(, 71c27480-8382-406f-a870-d2c3a3b36a18, 15, Finished, Available, Finished)

Gold table created: nyc_daily_revenue
Rows after cleaning: 109


SynapseWidget(Synapse.DataFrame, cc095b6b-9b13-46f0-85cc-6e3cc24c2219)

In [14]:
# Gold Metric 2: Avg Trip Distance per Month
from pyspark.sql.functions import col,count, sum,year,month,dayofmonth

monthly_distance = (
    df_silver
    .withColumn("year", year("pickup_datetime"))
    .withColumn("month", month("pickup_datetime"))
    .groupBy("year", "month")
    .agg(
        avg("trip_distance").alias("avg_trip_distance"),
        count("*").alias("total_trips")
    )
)

print(f"Rows after cleaning: {monthly_distance.count():,}")   
# Save as managed Delta table (Gold)
monthly_distance.write.format("delta").mode("overwrite").partitionBy("year","month").saveAsTable("nyc_monthly_distance")

print("Gold table created: nyc_monthly_distance")
display(monthly_distance.limit(20))

StatementMeta(, 71c27480-8382-406f-a870-d2c3a3b36a18, 16, Finished, Available, Finished)

Rows after cleaning: 12
Gold table created: nyc_monthly_distance


SynapseWidget(Synapse.DataFrame, 4f749737-7821-4c77-882b-2627897f3cd3)

In [15]:
payment_summary = (
    df_silver
    .groupBy("payment_type")
    .agg(
        count("*").alias("num_trips"),
        sum("fare_amount").alias("revenue")
    )
    .orderBy("payment_type")
)

# Save as managed Delta table
payment_summary.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("nyc_payment_summary")

print(f"Rows after cleaning: {payment_summary.count():,}")   
print("Gold table created: nyc_payment_summary")
# display(payment_summary)
spark.table("nyc_payment_summary").show()

StatementMeta(, 71c27480-8382-406f-a870-d2c3a3b36a18, 17, Finished, Available, Finished)

Rows after cleaning: 5
Gold table created: nyc_payment_summary
+------------+---------+-----------------+
|payment_type|num_trips|          revenue|
+------------+---------+-----------------+
|           1|   588753|       6740260.02|
|           2|   344467|3743815.540000001|
|           3|     7083|         81582.23|
|           4|     2434|         30603.32|
|           5|       12|            215.5|
+------------+---------+-----------------+

